In [1]:
# Import libraries
import pandas as pd
import math
from scipy.stats import norm

### Calculate annualized historical volatility
1. Calculate the daily rates using the natural log of daily return
$$x_t = ln(\frac{S_t}{S_{t-1}})$$
2. Calculate the sample mean of daily rates
$$\bar{x}=\frac{\sum_{i=0}^{n-1}x_i}{n-1}$$
3. Calculate deviations of daily rates
$$\sigma_t=(x_t-\bar{x})^2$$
4. Calculate the standard deviation
$$\sigma_{daily}=\sqrt{\frac{\sum_{i=0}^{n-1}\sigma_i}{n-1}}$$
5. Annualize the daily standard deviation
$$\sigma_{annual}=\sigma_{daily}\sqrt{n}$$
$$n=\text{number of days}$$

In [2]:
symbol = input("Ticker Symbol: ")
price_history = pd.read_csv(f"{symbol}.csv", skipinitialspace=True, usecols=["Adj Close"])
price_history["Daily Rate"] = None
price_history["Deviation"] = None

count = len(price_history.index)
for i in range(1, count):
    prev_price = price_history.loc[i - 1, "Adj Close"]
    curr_price = price_history.loc[i, "Adj Close"]
    price_history.loc[i, "Daily Rate"] = math.log(curr_price / prev_price)

sample_mean = price_history["Daily Rate"].fillna(0).sum() / (count - 1)

for i in range(1, count):
    rate = price_history.loc[i, "Daily Rate"]
    price_history.loc[i, "Deviation"] = pow(rate - sample_mean, 2)

sample_stdev = math.sqrt(price_history["Deviation"].fillna(0).sum() / (count - 1))
VOLATILITY = math.sqrt(count) * sample_stdev
print("Volatility:", VOLATILITY)

Ticker Symbol: AAPL
Volatility: 0.3546863540421135


In [3]:
# Additional Inputs
RISK_FREE = float(input("Risk-free Rate: "))
DIVIDEND_YIELD = float(input("Dividend Yield: "))
SPOT = float(input("Current Spot Price: "))

Risk-free Rate: 0.05
Dividend Yield: 0.0071
Current Spot Price: 129.93


In [4]:
# Black-Scholes Inputs
STRIKE = float(input("Strike Price: "))
EXPIRY = float(input("Time Until Expiration (yrs): "))

Strike Price: 120
Time Until Expiration (yrs): 1


### Black-Scholes formula
$$C=S_0e^{-rT}N(d_1)-Ke^{-rT}N(d_2)$$
$$\text{where }d_1=\frac{ln\frac{S_0}{K}+(r+\frac{\sigma^2}{2})T}{\sigma\sqrt{T}}$$
$$\text{and }d_2=d_1-\sigma\sqrt{T}$$
- $C=\text{European call price}$
- $N=\text{CDF of the normal distribution}$
- $S_0=\text{Current spot price}$
- $K=\text{Strike price}$
- $r=\text{risk-free interest rate}$
- $T=\text{Time until expiration}$
- $\sigma=\text{Volatility of asset}$

In [5]:
d1 = (math.log(SPOT / STRIKE) + (RISK_FREE - DIVIDEND_YIELD + 0.5 * pow(VOLATILITY, 2)) * EXPIRY) / (VOLATILITY * math.sqrt(EXPIRY))
d2 = d1 - VOLATILITY * math.sqrt(EXPIRY)
call = SPOT * math.exp(-DIVIDEND_YIELD * EXPIRY) * norm.cdf(d1) - STRIKE * math.exp(-RISK_FREE * EXPIRY) * norm.cdf(d2)
print(call)

25.54225025904725
